# **IP2_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment.

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01.
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r."""


In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Decision Variables:
Let's denote:

- X[i] as a binary variable capturing whether to invest in investment strategy i (1 if yes and 0 if no), for each i = 1, 2, ..., 10. 

Data Variables:
Given in the problem statement are:

- historical returns data from file "investments_data.csv" let it be represented as H[i] for each i = 1, 2, ..., 10.
- costs of each investment strategy be represented in vector A[i] = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] for i = 1, 2, ..., 10.
- budget scenarios and their probabilities be represented as, B[j] = [1000, 1100, 900, 1200] for each j = 1, 2, 3, 4 respectively and their associated probabilities as P[j] = [0.55, 0.4, 0.04, 0.01].
- acceptable risk tolerance level is given as r=2.


### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response2.choices[0].message.content)

Objective Function:

Given the above definitions, we aim to maximize our expected returns while taking into account the variability of the returns (risk). Therefore, we need to maximize the following objective function:

max Σ_i (H[i] * X[i]) - r * √(Σ_i (H[i]^2 * X[i]))

This objective function expresses our aim to maximize expected returns (Σ_i (H[i] * X[i])), penalized by the risk associated with the investment (std. dev. or variance of the returns) scaled with the risk tolerance factor r. The square root is used to transition from variance to standard deviation.

This objective function will be further constrained by the constraints representing the budget scenarios and their risks.


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response3.choices[0].message.content)

Constraints:

Our decision variables should be binary (either we invest in an asset or don't) so, 
for each i = 1, 2, ..., 10,
0 <= X[i] <= 1
and
X[i] ∈ {0,1}

Since we can invest only once in each investment:
for each i = 1, 2, ..., 10,
X[i] = 1

Now coming to budget uncertainty,
The budget available for each scenario j = 1, 2, 3, 4 should not be exceeded:
Σ_i (A[i] * X[i]) <= B[j]

The possibility of exceeding the budget in each scenario should not exceed the tolerable probability of 0.4. Given that the probabilities for each scenario are P[j] and since these probabilities sum up to 1, this requirement can be modeled as:
Σ_j (P[j] * Indicator(Σ_i (A[i] * X[i]) > B[j])) <= 0.4

The 'Indicator' function here represents an indicator that takes the value 1 if the investment is above the available budget and 0 otherwise. Normally this would be a difficult constraint to model mathematically directly in a standard mathematical programming model, but it could be achieved with a more sophisti

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response4.choices[0].message.content)

The following is a simple representation of the problem as noted above. Please note, the described problem is a mixed-integer nonlinear problem (MINLP), which would require a specific solver and fine tuning of the problem to ensure the problem's solvability.

```python
# Sample Pyomo model

from pyomo.environ import *

# Create model
model = ConcreteModel()

# Enumerate data
A = {1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225}
H = {i:(0.1*i) for i in A.keys()}   # Dummy data, replace with actual historical data
P = {1:0.55, 2:0.4, 3:0.04, 4:0.01}
B = {1:1000, 2:1100, 3:900, 4:1200}
r = 2

# Variables
model.X = Var(A.keys(), within=Binary)

# Objective
model.obj = Objective(expr=sum(H[i]*model.X[i] for i in model.X) - r*sqrt(sum(H[i]**2 * model.X[i] for i in model.X)), sense=maximize)

# Budget constraint
def budget_constraint(model, j):
    return sum(A[i]*model.X[i] for i in model.X) <= B[j]
model.budget_con = Constraint(P.keys(), rule=budget_constraint)

# Probabilistic risk constraint
# Pyomo does not easily handle indicator functions and chance constraints, so that part of the problem formulation needs to be refined

SolverFactory('couenne').solve(model)
model.pprint()
```

Note: This is a sample Pyomo model for the problem assuming 'couenne' solver and would not work for the actual problem as stated (especially with the probabilistic risk constraint). Pyomo doesn't directly handle chance constraints or stochastic programming within its standard framework. This code may need suitable modification to work for the actual problem. Also, 'Couenne' will not work unless it's installed appropriately on the machine. You can replace 'couenne' with any other suitable MINLP solver you have available. Also, replace the H and A arrays with the real data. And, handle chance constraint as per the necessity of the problem. This code provides a structure for the problem, not a complete solution.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.6 MB/s eta 0:00:00


In [ ]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Enumerate data
A = {1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225}
H = {i:(0.1*i) for i in A.keys()}   # Dummy data, replace with actual historical data
P = {1:0.55, 2:0.4, 3:0.04, 4:0.01}
B = {1:1000, 2:1100, 3:900, 4:1200}
r = 2

# Variables
model.X = Var(A.keys(), within=Binary)

# Objective
model.obj = Objective(expr=sum(H[i]*model.X[i] for i in model.X) - r*sqrt(sum(H[i]**2 * model.X[i] for i in model.X)), sense=maximize)

# Budget constraint
def budget_constraint(model, j):
    return sum(A[i]*model.X[i] for i in model.X) <= B[j]
model.budget_con = Constraint(P.keys(), rule=budget_constraint)

# Probabilistic risk constraint
# Pyomo does not easily handle indicator functions and chance constraints, so that part of the problem formulation needs to be refined

SolverFactory('knitro').solve(model)
model.pprint()

1 Var Declarations
    X : Size=10, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.1*X[1] + 0.2*X[2] + 0.30000000000000004*X[3] + 0.4*X[4] + 0.5*X[5] + 0.6000000000000001*X[6] + 0.7000000

In [ ]:
from pyomo.environ import *
import pandas as pd
# Create model
model = ConcreteModel()

# Enumerate data
A = {1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225}
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append(str((i+1)))
df.columns = headers
H = dict(df)
P = {1:0.55, 2:0.4, 3:0.04, 4:0.01}
B = {1:1000, 2:1100, 3:900, 4:1200}
r = 2

# Variables
model.X = Var(A.keys(), within=Binary)

# Objective
model.obj = Objective(expr=sum(H[i,j]*model.X[i] for i in model.X) - r*sqrt(sum(H[i,j]**2 * model.X[i] for i in model.X)), sense=maximize)

# Budget constraint
def budget_constraint(model, j):
    return sum(A[i]*model.X[i] for i in model.X) <= B[j]
model.budget_con = Constraint(P.keys(), rule=budget_constraint)

# Probabilistic risk constraint
# Pyomo does not easily handle indicator functions and chance constraints, so that part of the problem formulation needs to be refined

SolverFactory('knitro').solve(model)
model.pprint()

KeyError: 1

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**